Standardized environemnt:

    (1) In the top right click the select kernal.

    (2) Choose "Select another kernal"

    (3) Then "Python Environment"

    (4) Then "Create Python Environment"

    (5) Then "Conda" 

    (6) Then "3.12"

SSL Certifications:

    [1] Run the command "ls ~/.kaggle/kaggle.json"
        Make sure this is in Users/name/.kaggle/kaggle.json

    [2] run the command "pip install --upgrade certifi"

    [3] run the command "pip install --force-reinstall pygments"

    [4] restart vsCode

This will ensure everyone is working on the same python environment.

In [ ]:
# Run the following commands in your terminal before running this cell:
''' 1) Change your directory to week1:
        - open a terminal (ctr + `)
        - enter the command: cd week1
    2) install requirement by running
        - pip install -r requirements.txt'''
import os
import csv
import requests
import pandas as pd
import praw
import kaggle as kg
import nltk
import time
from nltk.sentiment import SentimentIntensityAnalyzer
from pytrends.request import TrendReq

Now, we need to go create our API keys/credentials.

Reddit.com -- 

    (1) Go to reddit.com and make an account.
    
    (2) Go to Settings -> Privacy -> Third party app authorizations

    (3) In the top left click create token.

    (4) Enter any name such as "pqtBitcoin":
        Select Script
        Enter any website in redirrect url (you can use google.com)

    (5) Copy the secret phrase and store it in the code block below.
        Copy the phrase under "personal use script" and copy it into your_client_id below

Kaggle.com --

    (1) Create an account

    (2) Navigate to "Account" → "Create New API Token"

    (3) Save this below

In [ ]:
# Reddit 
YOUR_CLIENT_ID = "rVTYVH2BwZDj4uGXKO3LXA"
YOUR_CLIENT_SERET = "BMspeVM0JZ3jAxhp3rlNwW-Za1lYRQ"

# Kaggle
KAGGLE_USERNAME = "charlespersm"
KAGGLE_KEY = "aceeb0736c999e0ecf7881408ba696e2"

Now we'll download our first dataset from kaggle.com

* Note that this file is intentionally too large to work with, next week we will focus on cleaning our data, but for now focus on the data collection part.


In [ ]:
## Dataset 1: Bitcoin Historical Data
'''Gets min/min bitcoin data since 2012'''
data_1 = "mczielinski/bitcoin-historical-data"
df_1 = "onzip"
kg.api.dataset_download_files(dataset = data_1, path=df_1, unzip=True)


## Here we download the dataset onto a zipfile.
    # You can see the dataset directly by clicking on the file on the left.


## Alternativly you can download it directly as a csv:
''' 
data_1 = "mczielinski/bitcoin-historical-data"
file_name = "btcusd_1-min_data.csv"
kaggle.api.dataset_download_file(dataset=data_1, file_name=file_name, path="./")
df_1 = pd.read_csv(file_name)
'''

In [ ]:
## We now can access this file directly and 'look' at it
df_1 = pd.read_csv("./onzip/btcusd_1-min_data.csv")

#.head(x) will give you the first x rows
df_1.head(100)
# feel free to try some other commands

Lets get one more historical dataset that compares BTC to other currencies
We will once again use kaggle for this so the code should look familiar.

In [ ]:
## Dataset 2: Bitcoin/Eth Prices
'''
    Gets day/day bitcoin data from start-Present
    *** Note there are some other useful datasets here 
    Explore!
'''
data_2 = "kapturovalexander/bitcoin-and-ethereum-prices-from-start-to-2023"
df_2 = "on_1.zip"
kg.api.dataset_download_files(dataset = data_2,path=df_2,unzip = True)
''' 
    There are 10 files in the zip: df_2 is currently 
    set to Eth/USD for the month of January, you can 
    change that by replacing on_1.zip/{filename.csv}
    with the name of the file.
'''
df_2 = pd.read_csv("on_1.zip/ETH-USD (01-05.2024).csv")

In [ ]:
## Look at the dataset
df_2.head(100)

## ALSO note that there are many other datasets in the zip folder on_1.zip
# you can use any of these with df = pd.read_csv("name.csv")

We now have historic data, lets try to get a fear and greed index

That will give us a normalized score of investor sentiment.

In [ ]:
## Dataset 3: Bitcoin F&G index
'''Uses live data to get the fear and greed index of bitcoin'''
## get the data (json)
def get_fng_data():
    url = "https://api.alternative.me/fng/"
    params = {"limit": 0, "format": "json", "date_format": "world"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["data"]
    else:
        print(response.status_code)
        return None
## json->csv
def convert_csv(data, filename):
    if not data:
        print("No data")
        return
    keys = data[0].keys()
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)


df_3 = get_fng_data()
convert_csv(df_3,"fear_greed_index.csv")
df_3 = pd.read_csv("./fear_greed_index.csv")

In [ ]:
## Run .head() to inspect the new dataset
df_3.head(100)

We will now scrape reddit to get any posts that might cause consumer sentiment to change:

This will require a bit more setup which we will do below


In [ ]:
## Reddit setup
auth = requests.auth.HTTPBasicAuth(YOUR_CLIENT_ID,YOUR_CLIENT_SERET)

## Fill in the {username} and {password} with your reddit username/password
data = {
    'grant_type': 'password',
    'username': 'princeton-pqt',
    'password': 'Charfyee3!123'
}

## This is an arbitrary name
headers = {'User-Agent': 'MyAPI/0.0.1'}

## We now get a token which lets us make requests 
res = requests.post('https://www.reddit.com/api/v1/access_token',auth=auth, data=data, headers=headers)
res.json()
TOKEN = res.json()['access_token']
headers['Authorization'] = f'bearer {TOKEN}'
print(TOKEN)

We now have the required credentials to do so!

In [ ]:
## Dataset 5: Reddit Scraping
''' Notation: https://oauth.reddit.com/r/Bitcoin/hot
    /Bitcoin - this is the subreddit we want you can change this as needed
    /hot - this is the type of post you can change this to something like /new if you want newer posts
    you can add other parameters such as params={'limit':100} which limits the number of posts to 100
'''
res = requests.get('https://oauth.reddit.com/r/Bitcoin/hot',headers=headers )
res.json()
for post in res.json()['data']['children'][:1]:
   print(post['data'])


post['data'].keys()

Above, we printed every post and it's subdata.

We also printed the keys which are the possible headers we can construct a dataset with

Lets now iterate through res and choose the ones we want


In [ ]:
df5 = []
# Loop through the JSON response and collect relevant fields
for post in res.json()['data']['children']:
    df5.append({  # Collect data correctly
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'ups': post['data']['ups'],
    })

# Create DataFrame from collected data
df_5 = pd.DataFrame(df5)  # Now it works!

# Save to CSV
df_5.to_csv('reddit_posts.csv', index=False)

In [ ]:
## Look at the dataset
df_5.head(100)

Our Final Datset will be google searches for Bitcoin.

In [ ]:
## Dataset 6: Google Search Trends
'''Maps google search trends with keyword 'Bitcoin' '''
def fetch_google(keyword, timeframe, max_retries=3):
    """Fetch Google Search Trends for a given keyword and timeframe, and save to a CSV file with additional insights."""
    
    retry_count = 0  # Track retries

    while retry_count < max_retries:
        try:
            print(f"Fetching Google Trends data for '{keyword}' with timeframe '{timeframe}' (Attempt {retry_count + 1})")

            # Initialize Pytrends with a User-Agent to bypass blocks
            pytrends = TrendReq(hl="en-US", tz=360, requests_args={'headers': {'User-Agent': 'Mozilla/5.0'}})

            # Build the search payload
            pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo="US", gprop="")

            # Fetch the interest over time
            trends_data = pytrends.interest_over_time()

            # Check if data is returned
            if trends_data.empty:
                print("No data returned. Check if the keyword and timeframe are valid.")
                return

            # Remove the "isPartial" column if it exists
            if "isPartial" in trends_data.columns:
                trends_data.drop(columns=["isPartial"], inplace=True)

            # 🔹 Add useful columns
            trends_data["Rolling_Avg"] = trends_data[keyword].rolling(window=7, min_periods=1).mean()  # 7-day moving avg
            trends_data["Normalized_Score"] = (trends_data[keyword] - trends_data[keyword].min()) / \
                                              (trends_data[keyword].max() - trends_data[keyword].min()) * 100  # Scale 0-100

            # Generate timestamped filename
            timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"google_trends_{keyword.lower()}_{timestamp}.csv"
            filepath = os.path.join(os.getcwd(), filename)

            # Save to CSV
            trends_data.to_csv(filepath)
            print(f"Data successfully saved to '{filepath}'")
            return  # Exit function on success

        except Exception as e:
            print(f"Error fetching Google Trends data: {e}")
            retry_count += 1
            wait_time = 5 * (2 ** retry_count)  # Exponential backoff
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)

# Fetch Google Trends data for Bitcoin over the last month
fetch_google("Bitcoin", "today 1-m")  # Last 30 days


In [ ]:
## Look at dataset
df_6 = pd.read_csv("google_trends_bitcoin_2025-03-02_16-07-28.csv")
df_6.head(20)

Thats it! We now have at least 6 datasets that we'll be able to get information for our models from

Ideally you're group will go find additional datasets that will make your project unique and more accurate

If you have any questions or if the code isn't working please contact me and I'll get back to you as soon as possible

cm6268 @ princeton.edu

